## Подключение библиотек и определение класса Dataset

In [15]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from decomposition import DecompositionLayer
torch.set_num_threads(9)

class MyDataset(TensorDataset):
    def __init__(self, data, window, output):
        self.data = data
        self.window = window
        self.output = output

    def __getitem__(self, index):
        x = self.data[index:index+self.window]
        y = self.data[index+self.window:index+self.window+self.output]
        return x, y

    def __len__(self):
        return len(self.data) - self.window - self.output

## Определение параметров модели

In [16]:
input_size = 15
pred = input_size
learning_rate = 0.0001
output_size = 20
data_size  = 4000
step = 10
m = 10 #на сколько шагов предсказать
data_set = 9000
model_name = "update_model_v2"

## Класс деокмпозиции временного ряда для получение сезонной и трендовой составляющей

In [17]:
class DecompositionLayer(nn.Module):
    """
    Returns the trend and the seasonal parts of the time series.
    """

    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=1, padding=0) # moving average 

    def forward(self, x):
        """Input shape: Batch x Time x EMBED_DIM"""
        # padding on the both ends of time series
        num_of_pads = (self.kernel_size) // 2
        front = x[:, 0:1, :].repeat(1, num_of_pads, 1)
        end = x[:, -1:, :].repeat(1, num_of_pads, 1)
        x_padded = torch.cat([front, x, end], dim=1)

        # calculate the trend and seasonal part of the series
        x_trend = self.avg(x_padded.permute(0, 2, 1)).permute(0, 2, 1)
        #print(x_trend.shape)
        x_seasonal = x - x_trend
        return x_seasonal, x_trend

## Класс DLinear и функция для обучения модели

In [18]:
class DLinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DLinearModel, self).__init__()
        self.linear_seasonal = nn.Linear(input_size, output_size)
        self.linear_trend = nn.Linear(input_size, output_size)
        self.decomposition = DecompositionLayer(input_size)
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, context):
        seasonal, trend = self.decomposition(context)
        #print(seasonal, trend)
        seasonal_output = self.linear_seasonal(seasonal.reshape(1, 1, -1))
        trend_output = self.linear_trend(trend.reshape(1, 1, -1))
        
        return seasonal_output + trend_output


def train_model(model, dataloader, criterion, optimizer, num_epochs=100):
    for epoch in range(num_epochs):
        print("Epoch = ", epoch)
        for X, Y in dataloader:
            
            optimizer.zero_grad()
            output = model.forward(X)
            loss = criterion(output, Y)
            loss.backward()
            optimizer.step()


In [19]:

data = pd.read_csv('ETTh1.csv')  
X = torch.tensor(data['HUFL'].values[:data_size:step], dtype=torch.float32).view(-1, 1)
x = pd.read_csv("ETTh1.csv").HUFL

In [20]:
model = DLinearModel(input_size, output_size)
criterion = nn.SmoothL1Loss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#TODO: переписать dataloader для правильно обучения

window_size = input_size  # Размер окна для rolling window forecasting
#переписать

dataset = MyDataset(X, window_size, output_size)

#print(x.rolling(window_size))

dataloader = DataLoader(dataset)#, shuffle=True)

In [21]:
len(dataloader)

365

In [22]:
train_model(model, dataloader, criterion, optimizer)


Epoch =  0


c:\Users\evgen\miniconda3\envs\MyDLinear\Lib\site-packages\torch\nn\modules\loss.py:933: UserWarning: Using a target size (torch.Size([1, 20, 1])) that is different to the input size (torch.Size([1, 1, 20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch =  1
Epoch =  2
Epoch =  3
Epoch =  4
Epoch =  5
Epoch =  6
Epoch =  7
Epoch =  8
Epoch =  9
Epoch =  10
Epoch =  11
Epoch =  12
Epoch =  13
Epoch =  14
Epoch =  15
Epoch =  16
Epoch =  17
Epoch =  18
Epoch =  19
Epoch =  20
Epoch =  21
Epoch =  22
Epoch =  23
Epoch =  24
Epoch =  25
Epoch =  26
Epoch =  27
Epoch =  28
Epoch =  29
Epoch =  30
Epoch =  31
Epoch =  32
Epoch =  33
Epoch =  34
Epoch =  35
Epoch =  36
Epoch =  37
Epoch =  38
Epoch =  39
Epoch =  40
Epoch =  41
Epoch =  42
Epoch =  43
Epoch =  44
Epoch =  45
Epoch =  46
Epoch =  47
Epoch =  48
Epoch =  49
Epoch =  50
Epoch =  51
Epoch =  52
Epoch =  53
Epoch =  54
Epoch =  55
Epoch =  56
Epoch =  57
Epoch =  58
Epoch =  59
Epoch =  60
Epoch =  61
Epoch =  62
Epoch =  63
Epoch =  64
Epoch =  65
Epoch =  66
Epoch =  67
Epoch =  68
Epoch =  69
Epoch =  70
Epoch =  71
Epoch =  72
Epoch =  73
Epoch =  74
Epoch =  75
Epoch =  76
Epoch =  77
Epoch =  78
Epoch =  79
Epoch =  80
Epoch =  81
Epoch =  82
Epoch =  83
Epoch =  84
E

In [23]:
torch.save(model.state_dict(), model_name)

In [24]:
model.load_state_dict(torch.load(model_name))
model.eval()

DLinearModel(
  (linear_seasonal): Linear(in_features=15, out_features=20, bias=True)
  (linear_trend): Linear(in_features=15, out_features=20, bias=True)
  (decomposition): DecompositionLayer(
    (avg): AvgPool1d(kernel_size=(15,), stride=(1,), padding=(0,))
  )
)

In [25]:
model.parameters

<bound method Module.parameters of DLinearModel(
  (linear_seasonal): Linear(in_features=15, out_features=20, bias=True)
  (linear_trend): Linear(in_features=15, out_features=20, bias=True)
  (decomposition): DecompositionLayer(
    (avg): AvgPool1d(kernel_size=(15,), stride=(1,), padding=(0,))
  )
)>

In [26]:
def predict_future_values(model, X_f, window_size, m):
    #predicted_values = initial_values.clone().detach().view(-1).tolist()
    predicted_values = []
    
    for i in range(m):
        
        
        #проверить что модель принимает нужный набор значений
        #last_X = torch.tensor([predicted_values[-1]], dtype=torch.float32).view(-1, 1)
        # last_X_t = torch.tensor([predicted_values[-1]], dtype=torch.float32).view(-1, 1)
        
        prediction = model(X_f)

        
        predicted_values.append(prediction.tolist()[-1][-1][-1])
        if(window_size > i):
            
            X_f = torch.tensor(data['HUFL'].values[data_size-window_size+i+1::step] + predicted_values, dtype=torch.float32).reshape(1, 1, -1)
        else:
            X_f = torch.tensor(predicted_values, dtype=torch.float32).reshape(1, 1, -1)
            
        

    return predicted_values

In [29]:
initial_values = torch.cat([X[-input_size:]]).reshape(1, 1, -1)

pred = input_size
X_f = torch.tensor(data['HUFL'].values[data_set-pred:data_set-pred+pred*step:step], dtype=torch.float32).view(-1, 1)
print(X_f)
dataset_f = MyDataset(X_f, pred, output_size)
print(dataset_f.data)
#print(x.rolling(window_size))

dataloader_f = DataLoader(dataset_f)#, shuffle=True)

X_f


tensor([[  5.8270],
        [  7.1000],
        [  9.5780],
        [ -7.5690],
        [  9.9800],
        [ -4.2870],
        [ 15.8740],
        [  9.3100],
        [  1.8750],
        [ 14.9370],
        [-10.2480],
        [ 13.8650],
        [  2.0760],
        [ 11.6550],
        [ 15.4720]])
tensor([[  5.8270],
        [  7.1000],
        [  9.5780],
        [ -7.5690],
        [  9.9800],
        [ -4.2870],
        [ 15.8740],
        [  9.3100],
        [  1.8750],
        [ 14.9370],
        [-10.2480],
        [ 13.8650],
        [  2.0760],
        [ 11.6550],
        [ 15.4720]])


tensor([[  5.8270],
        [  7.1000],
        [  9.5780],
        [ -7.5690],
        [  9.9800],
        [ -4.2870],
        [ 15.8740],
        [  9.3100],
        [  1.8750],
        [ 14.9370],
        [-10.2480],
        [ 13.8650],
        [  2.0760],
        [ 11.6550],
        [ 15.4720]])

In [30]:
X_t = X_f.tolist()
X_t

[[5.827000141143799],
 [7.099999904632568],
 [9.57800006866455],
 [-7.568999767303467],
 [9.979999542236328],
 [-4.2870001792907715],
 [15.87399959564209],
 [9.3100004196167],
 [1.875],
 [14.937000274658203],
 [-10.248000144958496],
 [13.864999771118164],
 [2.0759999752044678],
 [11.654999732971191],
 [15.472000122070312]]

In [35]:

predicted_values = []




    
X = torch.tensor([X_t])
prediction = model(X)
predicted_values = prediction.tolist()[-1][-1]

      
    


future_predictions = predicted_values

In [37]:
predicted_values

[9.157816886901855,
 4.127630233764648,
 3.435462474822998,
 3.3649544715881348,
 2.259443998336792,
 5.177505016326904,
 7.103431701660156,
 5.0743088722229,
 8.173192977905273,
 8.159629821777344,
 3.7868566513061523,
 8.918203353881836,
 6.028493881225586,
 4.300507545471191,
 2.959487199783325,
 7.764525413513184,
 9.798245429992676,
 6.328813552856445,
 6.913966655731201,
 8.430513381958008]

In [38]:
future_predictions

[9.157816886901855,
 4.127630233764648,
 3.435462474822998,
 3.3649544715881348,
 2.259443998336792,
 5.177505016326904,
 7.103431701660156,
 5.0743088722229,
 8.173192977905273,
 8.159629821777344,
 3.7868566513061523,
 8.918203353881836,
 6.028493881225586,
 4.300507545471191,
 2.959487199783325,
 7.764525413513184,
 9.798245429992676,
 6.328813552856445,
 6.913966655731201,
 8.430513381958008]

In [39]:
len(future_predictions)

20

Future Predictions: [9.157816886901855, 4.127630233764648, 3.435462474822998, 3.3649544715881348, 2.259443998336792, 5.177505016326904, 7.103431701660156, 5.0743088722229, 8.173192977905273, 8.159629821777344, 3.7868566513061523, 8.918203353881836, 6.028493881225586, 4.300507545471191, 2.959487199783325, 7.764525413513184, 9.798245429992676, 6.328813552856445, 6.913966655731201, 8.430513381958008]
10


In [41]:

print("Future Predictions:", future_predictions)
time = [data_set-output_size*step+i*step for i in range(2*output_size)]
print(m)
plt.plot(time, data['HUFL'].values[data_set-output_size*step:data_set+(output_size)*step:step])
#plt.plot(, )
pred = data['HUFL'].values[data_set-1]
time = [data_set+1+i*step for i in range(output_size)]

    
plt.plot(time, future_predictions, 'g--')

plt.show()



